# Module 2 - Programming Assignment

## Solving Normal Form Games

In [1]:
from IPython.core.display import *

An example of a normal form game, the Prisoner's Dilemma game is shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

This program implements a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game` and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It works with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return `None`.


### Helper Functions

**Remove column**

The remove_column function, given a game and a column index, removes that column from the game.  The removal of a column from a game, is the removal of a strategy for player 2.

In [3]:
def remove_column(game, column):
    for row in game:
        row.pop(column)
    return game

**Flip game**

The flip_game function inverts the list of lists so that the game is in a format that is easier to use to find the dominated strategy for player 2, specifically, so that one algorithm can be used to find the dominated strategy given any game.  This function makes it possible when finding a dominated strategy for any player, to compare each strategy row by row, as is done by default for player 1.

In [4]:
def flip_game(game):
    num_rows = len(game)
    row_length = len(game[0])
    
    flipped_game = []
    for i in range(row_length):
        column = []
        for j in range(num_rows):
            column.append(game[j][i])
        flipped_game.append(column)
            
    return flipped_game

**Get strategy indices**

Given a game, get_strategy_indices creates a list of lists of equal dimensions to the game, strategy_indices.  This list of lists is populated with the indices of each strategy.  As cells are removed from the game as the solve game function executes, the corresponding strategy index cells are removed from the strategy_indices list of lists.  If a solution is found by SEDS, there will be a single cell left in strategy_indices which contain the strategy indices of the Nash Equilibrium. 

In [5]:
def get_strategy_indices(game):
    strategy_indices = []
    for i in range(len(game)):
        row = game[i]
        strategy_row = []
        for j in range(len(row)):
            strategy_row.append((i, j))
    
        strategy_indices.append(strategy_row)
    
    return strategy_indices

**Check for one cell**

Given a game, the check_for_one_cell function determines whether there is only one cell left in the game.  This is used by the SEDS algorithm to determine whether a solution has been found.

In [6]:
def check_for_one_cell(game):
    if len(game) == 1 and len(game[0]) == 1:
        return True
    return False


**Remove strongly dominated strategy for player 1**

This algorithm finds a strongly dominated strategy for player 1 by comparing strategies one at a time until it finds one that dominates the other.  Once a dominated strategy is found, the corresponding cell is removed from the game and strategy_indices, and 'True' is returned.  If no dominated strategy is found, the function returns 'False' and does not remove any cells.  

In [7]:
def remove_strongly_dominated_strategy_player_1(game, strategy_indices):        
    for i in range(len(game) - 1):
        strategy1 = game[i]
        for j in range(len(game) - 1):
            j += 1
            strategy2 = game[j]
            one_dominates_two = True 
            two_dominates_one = True
            for k in range(len(strategy1)):
                value1 = strategy1[k][0]
                value2 = strategy2[k][0]
                one_dominates_two &= value1 > value2 
                two_dominates_one &= value2 > value1
            if one_dominates_two:
                game.pop(j)
                strategy_indices.pop(j)
                return True
            if two_dominates_one:
                game.pop(i)
                strategy_indices.pop(i)
                return True
            
    return False

**Remove strongly dominated strategy for player 2**

This algorithm finds a strongly dominated strategy for player 2 by comparing strategies one at a time until it finds one that dominates the other.  Once a dominated strategy is found, the corresponding cell is removed from the game and strategy_indices, and 'True' is returned.  If no dominated strategy is found, the function returns 'False' and does not remove any cells. This function is different than the above, because it flips the game before executing the algorithm using function flip_game. 

In [8]:
def remove_strongly_dominated_strategy_player_2(game, strategy_indices):    
    flipped_game = flip_game(game)
    for i in range(len(flipped_game) - 1):
        strategy1 = flipped_game[i]
        for j in range(len(flipped_game) - 1):
            j += 1
            strategy2 = flipped_game[j]
            one_dominates_two = True 
            two_dominates_one = True
            for k in range(len(strategy1)):
                value1 = strategy1[k][1]
                value2 = strategy2[k][1]
                one_dominates_two &= value1 > value2 
                two_dominates_one &= value2 > value1
            if one_dominates_two:
                remove_column(game, j)
                remove_column(strategy_indices, j)
                return True
            if two_dominates_one:
                remove_column(game, i)
                remove_column(strategy_indices, i)
                return True
    
    return False

**Initialize flags**

The initialize_flags function initializes various flags that are used in the algorithms to find weakly dominated strategies for player 1 and 2.  When comparing two strategies, to find whether strategy 1 dominates strategy 2, the flags keep track of whether strategy one dominates strategy two, there is at least one value in strategy 1 that is greater than the corresponding value in strategy 2, and there is at least one value in strategy 1 that is equal to the corresponding value in strategy 2.  These checks ensure that a dominated strategy follows all the criteria for being a weakly dominated strategy.

In [9]:
def initialize_flags():
    one_dominates_two = True 
    one_greater_flag = False
    two_dominates_one = True
    two_greater_flag = False
    equal_flag = False
    
    return one_dominates_two, one_greater_flag, two_dominates_one, two_greater_flag, equal_flag

**Update flags**

As the algorithms are executed to find weakly dominated strategies, the above described flags must be updated accordingly.  This function updates the flags according to a passed in value for strategy 1 and a passed in value for strategy 2.

In [10]:
def update_flags(value1, value2, one_greater_flag, two_greater_flag, equal_flag):
    if (value1 > value2):
        one_greater_flag = True

    if (value2 > value1):
        two_greater_flag = True

    if (value1 == value2):
        equal_flag = True
    
    return one_greater_flag, two_greater_flag, equal_flag

**Remove weakly dominated strategy for player 1**

This algorithm finds a weakly dominated strategy for player 1 by comparing strategies one at a time until it finds one that dominates the other.  Once a dominated strategy is found, the corresponding cell is removed from the game and strategy_indices, and 'True' is returned.  If no dominated strategy is found, the function returns 'False' and does not remove any cells. 

In [11]:

def remove_weakly_dominated_strategy_player_1(game, strategy_indices):        
    for i in range(len(game) - 1):
        strategy1 = game[i]
        for j in range(len(game) - 1):
            j += 1
            strategy2 = game[j]            
            one_dominates_two, one_greater_flag, two_dominates_one, two_greater_flag, equal_flag = initialize_flags()
            for k in range(len(strategy1)):
                value1 = strategy1[k][0]
                value2 = strategy2[k][0]
                one_dominates_two &= value1 >= value2 
                two_dominates_one &= value2 >= value1
                
                one_greater_flag, two_greater_flag, equal_flag = update_flags(value1, value2, one_greater_flag, two_greater_flag, equal_flag)
                
            if one_dominates_two and one_greater_flag and equal_flag:
                game.pop(j)
                strategy_indices.pop(j)
                return True
            
            if two_dominates_one and two_greater_flag and equal_flag:
                game.pop(i)
                strategy_indices.pop(i)
                return True
    
    return False

**Remove weakly dominated strategy for player 2**

This algorithm finds a weakly dominated strategy for player 2 by comparing strategies one at a time until it finds one that dominates the other.  Once a dominated strategy is found, the corresponding cell is removed from the game and strategy_indices, and 'True' is returned.  If no dominated strategy is found, the function returns 'False' and does not remove any cells. This function is different than the above, because it flips the game before executing the algorithm using function flip_game. 

In [12]:
def remove_weakly_dominated_strategy_player_2(game, strategy_indices):    
    flipped_game = flip_game(game)
    for i in range(len(flipped_game) - 1):
        strategy1 = flipped_game[i]
        
        for j in range(len(flipped_game) - 1):
            j += 1
            strategy2 = flipped_game[j]            
            one_dominates_two, one_greater_flag, two_dominates_one, two_greater_flag, equal_flag = initialize_flags()
            for k in range(len(strategy1)):
                value1 = strategy1[k][1]
                value2 = strategy2[k][1]
                one_dominates_two &= value1 >= value2 
                two_dominates_one &= value2 >= value1
                
                one_greater_flag, two_greater_flag, equal_flag = update_flags(value1, value2, one_greater_flag, two_greater_flag, equal_flag)
                
            if one_dominates_two and one_greater_flag and equal_flag:
                game.pop(j)
                strategy_indices.pop(j)
                return True
            
            if two_dominates_one and two_greater_flag and equal_flag:
                game.pop(i)
                strategy_indices.pop(i)
                return True
    
    return False

**Determine whether to continue elimination**

The function determine_continue_elimination is given boolean values that tell whether a weak or strong strategy can be removed for player 1 or 2.  This function is used to stop the solve game algorithm.  If no strategies can be eliminated by either player, then the algorithm should be stopped.  



In [13]:
def determine_continue_elimination(player_1_strong_removed, player_1_weak_removed, player_2_strong_removed, player_2_weak_removed):
    continue_elimination = False
    continue_elimination |= player_1_strong_removed 
    continue_elimination |= player_1_weak_removed 
    continue_elimination |= player_2_strong_removed 
    continue_elimination |= player_2_weak_removed
        
    return continue_elimination


---

### Solve Game
The solve_game function uses the above described helper functions to execute the SEDS algorithm:

Starting with either player it completes the following steps
1. Identify a dominated strategy and remove it from the game
2. Switch to the other player and remove a dominated strategy from the game
3. Continue until no player can eliminate a dominated strategy or a solution is found

If there is no strategy to eliminate, and the game is not down to a single cell, SEDS cannot find the Nash Equilibrium

The first function parameter is the game, in format as described above.  There is a second parameter, defaulted to False, which, if True causes the algorithm to consider weakly dominated strategies in addition to strongly dominated strategies.  If False, only strongly dominated strategies are considered

In [14]:
def solve_game(game, weak=False):
    strategy_indices = get_strategy_indices(game)
    continue_elimination = True
    player_1_weak_removed = False
    player_2_weak_removed = False
    
    while not check_for_one_cell(game) and continue_elimination:        
        player_1_strong_removed = remove_strongly_dominated_strategy_player_1(game, strategy_indices)
        if not player_1_strong_removed and weak:
            player_1_weak_removed = remove_weakly_dominated_strategy_player_1(game, strategy_indices)
        
        player_2_strong_removed = remove_strongly_dominated_strategy_player_2(game, strategy_indices)
        if not player_2_strong_removed and weak:
            player_2_weak_removed = remove_weakly_dominated_strategy_player_2(game, strategy_indices)
            
        continue_elimination = determine_continue_elimination(player_1_strong_removed, player_1_weak_removed, player_2_strong_removed, player_2_weak_removed)
    
    if not check_for_one_cell(game):
        return None
        
    return strategy_indices[0][0]

-----


In order to test the solve_game function, the following describes three (3) test cases, each of which is a 3x3 two player game. 

### Test Game 1. a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

Player 1 / Player 2  | 0 | 1 | 2
---- | ---- | ----
0  | 50 / 50 | 57 / 55 | 57 / 60
1  | 55 / 57 | 63 / 63 | 70 / 60
2  | 60 / 57 | 60 / 70 | 69 / 69

**Solution:** (1, 1)

In [15]:
test_game_1 = [ 
 [ (50, 50), (57, 55), (57, 60) ],
 [ (55, 57), (63, 63), (70, 60) ],
 [ (60, 57), (60, 70), (69, 69) ] ]

solution = solve_game( test_game_1)

In [16]:
assert solution == (1, 1)

### Test Game 2. a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

Player 1 / Player 2  | 0 | 1 | 2
---- | ---- | ----
0  | 50 / 50 | 50 / 50 | 50 / 60
1  | 55 / 57 | 63 / 63 | 70 / 60
2  | 50 / 57 | 60 / 70 | 69 / 69

**Solution:** (1, 1)

In [17]:
test_game_2 = [ 
 [ (50, 50), (50, 50), (50, 50) ],
 [ (50, 57), (63, 63), (70, 60) ],
 [ (50, 57), (60, 70), (69, 69) ] ]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)

In [18]:
assert strong_solution == None
assert weak_solution == (1, 1)

### Test Game 3. a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

Player 1 / Player 2  | 0 | 1 | 2
---- | ---- | ----
0  | 50 / 50 | 50 / 45 | 49 / 45
1  | 55 / 57 | 49 / 49 | 70 / 60
2  | 60 / 57 | 49 / 70 | 69 / 69

**Solution:** None

In [19]:
test_game_3 = [ 
 [ (50, 50), (50, 45), (49, 45) ],
 [ (50, 57), (35, 49), (70, 60) ],
 [ (60, 57), (49, 70), (69, 69) ] ]

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [20]:
assert strong_solution == None
assert weak_solution == None